In [ ]:
import kitchen.ingredients as k
import dropkick as dk
from MILWRM.ST import *
from MILWRM.MxIF import *
import sys; sys.path.append("../resources/")
from visium_utils import deconvolve_cnmf

# some stuff to make this notebook work better with Scanpy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from skimage import data, transform, io
from napari.utils.transforms import Affine

In [ ]:
# make output directories
import os
if not os.path.exists("ST_out"):
    os.mkdir("ST_out")

if not os.path.exists("ST_out/plots_overview/"):
    os.mkdir("ST_out/plots_overview/")
    
if not os.path.exists("ST_out/plots_overview/wholeslide/"):
    os.mkdir("ST_out/plots_overview/wholeslide/")

In [ ]:
sc.set_figure_params(transparent=True, dpi_save=400)
sc.settings.figdir = "ST_out/plots_overview/wholeslide/"
import seaborn as sns; sns.set_style("white")

---
# WD86056 Sample 6723-KL-1

In [ ]:
a1 = sc.read("../data/ST/6723_1_WD86056_master.h5ad"); a1

In [ ]:
WD86056 = img.from_npz(file="../data/MxIF/WD86056/WD86056_downsample8.npz"); WD86056

In [ ]:
img1 = io.imread("../data/MxIF/WD86056/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
img1 = img1[1::,:WD86056.img.shape[1]:,:]

In [ ]:
WD86056_HE = img(img1, mask=WD86056.mask)
WD86056_HE.scale()
#WD86056_HE.img = WD86056_HE.img[::-1,::-1,:]
#WD86056_HE.mask = WD86056_HE.mask[::-1,::-1]

In [ ]:
p = WD86056_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT71397_WD86056_histology.png")

In [ ]:
WD86056_HE

In [ ]:
bg_img = WD86056_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD86056_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/WD86056/WD86056_downsample8_6723_1_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/WD86056/WD86056_downsample8_6723_1_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
shape1 * affine1[0,0]

In [ ]:
scalef_adata = a1.uns["spatial"]["6723_KL_1"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef = affine1[0,0]

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - (WD86056_HE.img.shape[1] / (scalef_adata/scalef))
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + (WD86056_HE.img.shape[0] / (scalef/scalef_adata))

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - ((a1.uns["spatial"]["6723_KL_1"]["images"]["hires"].shape[1] / scalef_adata) - (WD86056_HE.img.shape[1] / scalef))
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + ((a1.uns["spatial"]["6723_KL_1"]["images"]["hires"].shape[0] / scalef_adata) - (WD86056_HE.img.shape[0] / scalef))

In [ ]:
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 500

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * (2*scalef_adata/scalef)

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="cnv_score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD86056_HE.img.shape[1],0,WD86056_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD86056 Sample 6723-KL-2

In [ ]:
a2 = sc.read("6723_2_WD86056_master.h5ad"); a2

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/WD86056/WD86056_downsample8_6723_2_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/WD86056/WD86056_downsample8_6723_2_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
scalef_adata2 = a2.uns["spatial"]["6723_KL_2"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef2 = affine2[0,0]

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - (WD86056_HE.img.shape[1] / (scalef_adata/scalef))
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + (WD86056_HE.img.shape[0] / (scalef/scalef_adata))

In [ ]:
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - ((a2.uns["spatial"]["6723_KL_2"]["images"]["hires"].shape[1] / scalef_adata) - (WD86056_HE.img.shape[1] / scalef))
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 2*((a2.uns["spatial"]["6723_KL_2"]["images"]["hires"].shape[0] / scalef_adata2) - (WD86056_HE.img.shape[0] / scalef2))

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 1500
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 500

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * (2*scalef_adata2/scalef2)

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 1500
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 500
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 500
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 200
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 20
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 500

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD86056_HE.img.shape[1],0,WD86056_HE.img.shape[0]],
    frameon=False
)

---
# WD86056 Sample 6723-KL-3

In [ ]:
a3 = sc.read("6723_3_WD86056_master.h5ad"); a3

In [ ]:
affine3 = np.genfromtxt(
    "../data/MxIF/WD86056/WD86056_downsample8_6723_3_affine.csv",
    delimiter=",",
)
shape3 = np.genfromtxt(
    "../data/MxIF/WD86056/WD86056_downsample8_6723_3_shape.csv",
    delimiter=",",
)

In [ ]:
affine3

In [ ]:
shape3

In [ ]:
scalef_adata3 = a3.uns["spatial"]["6723_KL_3"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef3 = affine1[0,0]

In [ ]:
a3.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a3.obsm["spatial"]),
        np.zeros((a3.obsm["spatial"].shape[0],1))
    ]),
    affine3.T
)[:,:2]

In [ ]:
a3.obsm["spatial_ws"]

In [ ]:
(WD86056_HE.img.shape[1] / (scalef_adata3/scalef3))

In [ ]:
(WD86056_HE.img.shape[0] / (scalef_adata3/scalef3))

In [ ]:
a3.obsm["spatial_ws"][:,0] = a3.obsm["spatial_ws"][:,0] - (WD86056_HE.img.shape[1] / (scalef_adata3/scalef3))
a3.obsm["spatial_ws"][:,1] = a3.obsm["spatial_ws"][:,1] + (WD86056_HE.img.shape[0] / (scalef3/scalef_adata3))

In [ ]:
((a3.uns["spatial"]["6723_KL_3"]["images"]["hires"].shape[0] / scalef_adata3) - (WD86056_HE.img.shape[0] / scalef3))

In [ ]:
((a3.uns["spatial"]["6723_KL_3"]["images"]["hires"].shape[1] / scalef_adata3) - (WD86056_HE.img.shape[1] / scalef3))

In [ ]:
a3.obsm["spatial_ws"][:,0] = a3.obsm["spatial_ws"][:,0] + 2*((a3.uns["spatial"]["6723_KL_3"]["images"]["hires"].shape[1] / scalef_adata3) - (WD86056_HE.img.shape[1] / scalef3))
a3.obsm["spatial_ws"][:,1] = a3.obsm["spatial_ws"][:,1] + 2*((a3.uns["spatial"]["6723_KL_3"]["images"]["hires"].shape[0] / scalef_adata3) - (WD86056_HE.img.shape[0] / scalef3))

In [ ]:
a3.obsm["spatial_ws"] = a3.obsm["spatial_ws"] * (2*scalef_adata3/scalef3)

In [ ]:
a3.obsm["spatial_ws"]

In [ ]:
a3.obsm["spatial_ws"][:,1] = a3.obsm["spatial_ws"][:,1] + 1300
a3.obsm["spatial_ws"][:,0] = a3.obsm["spatial_ws"][:,0] + 1300

In [ ]:
sc.pl.spatial(
    a3,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD86056_HE.img.shape[1],0,WD86056_HE.img.shape[0]],
    frameon=False
)

---
# WD86056 Combined

In [ ]:
a_ws = a1.concatenate([a2,a3], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("PAT71397_WD86056_combined_ws.h5ad", compression="gzip")

---
### Replace old `.obs` metadata with new

In [ ]:
a_ws = sc.read("PAT71397_WD86056_combined_ws.h5ad")

In [ ]:
del a_ws.obs #.drop(columns=['A1_SM', 'A2_MUC', 'A3_Adaptive', 'A4_Endocrine', 'A5_Cytotoxic', 'A6_DPEP1', 'A7_ABS', 'A8_Innate'], inplace=True)

In [ ]:
a1 = sc.read("6723_1_WD86056_master.h5ad")
a2 = sc.read("6723_2_WD86056_master.h5ad")
a3 = sc.read("6723_3_WD86056_master.h5ad")

In [ ]:
a = a1.concatenate([a2,a3], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["cnv_leiden"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD86056_HE.img.shape[1],0,WD86056_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT71397_WD86056_cnv_leiden.png"
)

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=signatures,
    img=bg_img,
    ncols=5,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD86056_HE.img.shape[1],0,WD86056_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT71397_WD86056_signatures.png"
)

In [ ]:
et = ['A1_Innate', 'A2_Cytotoxic', 'A3_Endocrine', 'A4_MUC', 'A5_ABS', 'A6_SM', 'A7_Adaptive', 'A8_Stem']

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=et,
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD86056_HE.img.shape[1],0,WD86056_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT71397_WD86056_archetypes.png",
    ncols=4,
)

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["CytoTRACE","cnv_score","cnv_leiden"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD86056_HE.img.shape[1],0,WD86056_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT71397_WD86056_cnv_progression.png",
)

---
# WD86055 Sample 6723-KL-4

In [ ]:
a4 = sc.read("6723_4_WD86055_master.h5ad"); a4

In [ ]:
WD86055 = img.from_npz(file="../data/MxIF/WD86055/WD86055_downsample8.npz"); WD86055

In [ ]:
img1 = io.imread("../data/MxIF/WD86055/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
WD86055_HE = img(img1, mask=WD86055.mask)
WD86055_HE.scale()
WD86055_HE.img = WD86055_HE.img[::-1,::-1,:]
WD86055_HE.mask = WD86055_HE.mask[::-1,::-1]

In [ ]:
p = WD86055_HE.show(RGB=True)

In [ ]:
bg_img = WD86055_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD86055_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
affine4 = np.genfromtxt(
    "../data/MxIF/WD86055/WD86055_downsample8_6723_4_affine.csv",
    delimiter=",",
)
shape4 = np.genfromtxt(
    "../data/MxIF/WD86055/WD86055_downsample8_6723_4_shape.csv",
    delimiter=",",
)

In [ ]:
affine4

In [ ]:
shape4

In [ ]:
scalef_adata4 = a4.uns["spatial"]["6723_KL_4"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef4 = affine4[0,0]

In [ ]:
scalef4

In [ ]:
scalef_adata4

In [ ]:
a4.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a4.obsm["spatial"]),
        np.zeros((a4.obsm["spatial"].shape[0],1))
    ]),
    affine4.T
)[:,:2]

In [ ]:
a4.obsm["spatial_ws"]

In [ ]:
#a4.obsm["spatial_ws"][:,0] = a4.obsm["spatial_ws"][:,0] - ((a4.uns["spatial"]["6723_KL_4"]["images"]["hires"].shape[1] / scalef_adata4) - (WD86055_HE.img.shape[1] / scalef4))
a4.obsm["spatial_ws"][:,1] = a4.obsm["spatial_ws"][:,1] + ((a4.uns["spatial"]["6723_KL_4"]["images"]["hires"].shape[0] / scalef_adata4) - (WD86055_HE.img.shape[0] / scalef4))

In [ ]:
a4.obsm["spatial_ws"] = a4.obsm["spatial_ws"] * (scalef_adata4/scalef4)

In [ ]:
a4.obsm["spatial_ws"][:,1] = a4.obsm["spatial_ws"][:,1] + 1300
#a4.obsm["spatial_ws"][:,0] = a4.obsm["spatial_ws"][:,0] + 1300

In [ ]:
a4.obsm["spatial_ws"][:,1] = a4.obsm["spatial_ws"][:,1] + 500
a4.obsm["spatial_ws"][:,0] = a4.obsm["spatial_ws"][:,0] - 300

In [ ]:
a4.obsm["spatial_ws"][:,1] = a4.obsm["spatial_ws"][:,1] + 500
a4.obsm["spatial_ws"][:,0] = a4.obsm["spatial_ws"][:,0] - 300

In [ ]:
a4.obsm["spatial_ws"][:,1] = a4.obsm["spatial_ws"][:,1] + 300
a4.obsm["spatial_ws"][:,0] = a4.obsm["spatial_ws"][:,0] - 300

In [ ]:
a4.obsm["spatial_ws"][:,1] = a4.obsm["spatial_ws"][:,1] + 300
a4.obsm["spatial_ws"][:,0] = a4.obsm["spatial_ws"][:,0] - 100

In [ ]:
#a4.obsm["spatial_ws"][:,1] = a4.obsm["spatial_ws"][:,1] + 300
a4.obsm["spatial_ws"][:,0] = a4.obsm["spatial_ws"][:,0] - 100

In [ ]:
a4.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a4,
    basis="spatial_ws",
    color="cnv_score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD86055_HE.img.shape[1],0,WD86055_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

In [ ]:
# flip coords in second sample
a4.obsm["spatial_ws"][:,0] = WD86055_HE.img.shape[1] - a4.obsm["spatial_ws"][:,0]
a4.obsm["spatial_ws"][:,1] = WD86055_HE.img.shape[0] - a4.obsm["spatial_ws"][:,1]

In [ ]:
# flip image back
WD86055_HE.img = WD86055_HE.img[::-1,::-1,:]
WD86055_HE.mask = WD86055_HE.mask[::-1,::-1]

bg_img = WD86055_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD86055_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = WD86055_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT71397_WD86055_histology.png")

In [ ]:
a4.write("PAT71397_WD86055_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a4 = sc.read("PAT71397_WD86055_combined_ws.h5ad")

In [ ]:
del a4.obs

In [ ]:
a4_normal = sc.read("6723_4_WD86055_master.h5ad")

In [ ]:
a4.obs = a4.obs.merge(a4_normal.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a4,
    basis="spatial_ws",
    color=["cnv_leiden"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD86055_HE.img.shape[1],0,WD86055_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    #save="_PAT71397_WD86055_cnv_leiden.png"
)

---
# WD87482 Sample 7003-AS-7

In [ ]:
a1 = sc.read("7003_7_WD87482_master.h5ad"); a1

In [ ]:
WD87482 = img.from_npz(file="../data/MxIF/WD87482/WD87482_downsample8.npz"); WD87482

In [ ]:
img1 = io.imread("../data/MxIF/WD87482/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
img1 = img1[1::,:WD87482.img.shape[1]:,:]

In [ ]:
WD87482_HE = img(img1, mask=WD87482.mask)
WD87482_HE.scale()
WD87482_HE.img = WD87482_HE.img[::-1,::-1,:]
WD87482_HE.mask = WD87482_HE.mask[::-1,::-1]

In [ ]:
WD87482_HE

In [ ]:
bg_img = WD87482_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD87482_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = WD87482_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT73899_histology.png")

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/WD87482/WD87482_downsample8_7003_7_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/WD87482/WD87482_downsample8_7003_7_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
shape1 * affine1[0,0]

In [ ]:
scalef_adata1 = a1.uns["spatial"]["7003_AS_7"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef1 = affine1[0,0]

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * (2*scalef_adata1/scalef1)

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 1000
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] - 800

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 500
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] - 100

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 300
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 200

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 200
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 200

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="cnv_score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87482_HE.img.shape[1],0,WD87482_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD87482 Sample 7003-AS-8

In [ ]:
a2 = sc.read("7003_8_WD87482_master.h5ad"); a2

In [ ]:
WD87482 = img.from_npz(file="../data/MxIF/WD87482/WD87482_downsample8.npz"); WD87482

In [ ]:
img1 = io.imread("../data/MxIF/WD87482/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
img1 = img1[1::,:WD87482.img.shape[1]:,:]

In [ ]:
WD87482_HE = img(img1, mask=WD87482.mask)
WD87482_HE.scale()
#WD87482_HE.img = WD87482_HE.img[::-1,::-1,:]
#WD87482_HE.mask = WD87482_HE.mask[::-1,::-1]

In [ ]:
WD87482_HE

In [ ]:
bg_img = WD87482_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD87482_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = WD87482_HE.show(RGB=True, mask_out=False)#, save_to="figures/show_PAT73899_histology.png")

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/WD87482/WD87482_downsample8_7003_8_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/WD87482/WD87482_downsample8_7003_8_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
scalef_adata2 = a2.uns["spatial"]["7003_AS_8"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef2 = affine2[0,0]

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * (2*scalef_adata2/scalef2)

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 500
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 1500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 50
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 1500

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87482_HE.img.shape[1],0,WD87482_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD87482 Combined

In [ ]:
# flip coords in second sample
a2.obsm["spatial_ws"][:,0] = WD87482_HE.img.shape[1] - a2.obsm["spatial_ws"][:,0]
a2.obsm["spatial_ws"][:,1] = WD87482_HE.img.shape[0] - a2.obsm["spatial_ws"][:,1]

In [ ]:
a_ws = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("PAT73899_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a_ws = sc.read("PAT73899_combined_ws.h5ad")

In [ ]:
del a_ws.obs

In [ ]:
a1 = sc.read("7003_7_WD87482_master.h5ad")
a2 = sc.read("7003_8_WD87482_master.h5ad")

In [ ]:
a = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["cnv_leiden"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87482_HE.img.shape[1],0,WD87482_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT73899_cnv_leiden.png"
)

---
# WD87480 Sample 7003-AS-5

In [ ]:
a1 = sc.read("7003_5_WD87480_master.h5ad"); a1

In [ ]:
WD87480 = img.from_npz(file="../data/MxIF/WD87480/WD87480_downsample8.npz"); WD87480

In [ ]:
img1 = io.imread("../data/MxIF/WD87480/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
img1 = img1[1::,:WD87480.img.shape[1]:,:]

In [ ]:
WD87480_HE = img(img1, mask=WD87480.mask)
WD87480_HE.scale()
WD87480_HE.img = WD87480_HE.img[::-1,::-1,:]
WD87480_HE.mask = WD87480_HE.mask[::-1,::-1]

In [ ]:
WD87480_HE

In [ ]:
bg_img = WD87480_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD87480_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = WD87480_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT71662_histology.png")

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/WD87480/WD87480_downsample8_7003_5_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/WD87480/WD87480_downsample8_7003_5_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
scalef_adata1 = a1.uns["spatial"]["7003_AS_5"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef1 = affine1[0,0]

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * (1.45*scalef_adata1/scalef1)

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 950
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 500

In [ ]:
#a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 950
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 100

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 50
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 100

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="pathology_annotation",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87480_HE.img.shape[1],0,WD87480_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD87480 Sample 7003-AS-6

In [ ]:
a2 = sc.read("7003_6_WD87480_master.h5ad"); a2

In [ ]:
img1 = io.imread("../data/MxIF/WD87480/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
img1 = img1[1::,:WD87480.img.shape[1]:,:]

In [ ]:
WD87480_HE = img(img1, mask=WD87480.mask)
WD87480_HE.scale()
#WD87480_HE.img = WD87480_HE.img[::-1,::-1,:]
#WD87480_HE.mask = WD87480_HE.mask[::-1,::-1]

In [ ]:
WD87480_HE

In [ ]:
bg_img = WD87480_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD87480_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = WD87480_HE.show(RGB=True, mask_out=False)#, save_to="figures/show_WD87480_histology.png")

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/WD87480/WD87480_downsample8_7003_6_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/WD87480/WD87480_downsample8_7003_6_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
scalef_adata2 = a2.uns["spatial"]["7003_AS_6"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef2 = affine2[0,0]

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * (1.45*scalef_adata2/scalef2)

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 500
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 500

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 500
#a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 500

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 50
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 50

In [ ]:
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 50
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 200

In [ ]:
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 50
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 100

In [ ]:
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 50
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 50

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 50
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 50

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
WD87480_HE.img.shape

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87480_HE.img.shape[1],0,WD87480_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD87480 Combined

In [ ]:
# flip coords in second sample
a2.obsm["spatial_ws"][:,0] = WD87480_HE.img.shape[1] - a2.obsm["spatial_ws"][:,0]
a2.obsm["spatial_ws"][:,1] = WD87480_HE.img.shape[0] - a2.obsm["spatial_ws"][:,1]

In [ ]:
img1 = io.imread("../data/MxIF/WD87480/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
img1 = img1[1::,:WD87480.img.shape[1]:,:]

In [ ]:
WD87480_HE = img(img1, mask=WD87480.mask)
WD87480_HE.scale()
WD87480_HE.img = WD87480_HE.img[::-1,::-1,:]
WD87480_HE.mask = WD87480_HE.mask[::-1,::-1]

In [ ]:
WD87480_HE

In [ ]:
bg_img = WD87480_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD87480_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
a_ws = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("PAT71662_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a_ws = sc.read("PAT71662_combined_ws.h5ad")

In [ ]:
del a_ws.obs

In [ ]:
a1 = sc.read("7003_5_WD87480_master.h5ad")
a2 = sc.read("7003_6_WD87480_master.h5ad")

In [ ]:
a = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["cnv_leiden"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87480_HE.img.shape[1],0,WD87480_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT71662_cnv_leiden.png"
)

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=signatures,
    ncols=5,
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87480_HE.img.shape[1],0,WD87480_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT71662_signatures.png"
)

In [ ]:
et = ['A1_Innate', 'A2_Cytotoxic', 'A3_Endocrine', 'A4_MUC', 'A5_ABS', 'A6_SM', 'A7_Adaptive', 'A8_Stem']

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=et,
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87480_HE.img.shape[1],0,WD87480_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT71662_archetypes.png",
    ncols=4,
)

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["CytoTRACE","cnv_score","cnv_leiden"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87480_HE.img.shape[1],0,WD87480_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT71662_cnv_progression.png",
)

---
# WD85806 Sample 7003-AS-3

In [ ]:
a1 = sc.read("7003_3_WD85806_master.h5ad"); a1

In [ ]:
WD85806 = img.from_npz(file="../data/MxIF/WD85806/WD85806_downsample8.npz"); WD85806

In [ ]:
img1 = io.imread("../data/MxIF/WD85806/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
img1 = img1[1::,:WD85806.img.shape[1]:,:]

In [ ]:
WD85806_HE = img(img1, mask=WD85806.mask)
WD85806_HE.scale()
WD85806_HE.img = WD85806_HE.img[::-1,::-1,:]
WD85806_HE.mask = WD85806_HE.mask[::-1,::-1]

In [ ]:
WD85806_HE

In [ ]:
bg_img = WD85806_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD85806_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = WD85806_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT73458_histology.png")

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/WD85806/WD85806_downsample8_7003_3_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/WD85806/WD85806_downsample8_7003_3_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
scalef_adata1 = a1.uns["spatial"]["7003_AS_3"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef_adata1

In [ ]:
scalef1 = affine1[0,0]

In [ ]:
scalef1

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * (scalef_adata1*1.9)#(scalef_adata1/scalef1)

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 200
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 7000

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 300
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 7000

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 200
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] - 100

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 50
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] - 100

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 50
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] - 50

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="pathology_annotation",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD85806_HE.img.shape[1],0,WD85806_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD85806 Sample 7003-AS-4

In [ ]:
a2 = sc.read("7003_4_WD85806_master.h5ad"); a2

In [ ]:
img1 = io.imread("../data/MxIF/WD85806/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
img1 = img1[1::,:WD85806.img.shape[1]:,:]

In [ ]:
WD85806_HE = img(img1, mask=WD85806.mask)
WD85806_HE.scale()
#WD87480_HE.img = WD87480_HE.img[::-1,::-1,:]
#WD87480_HE.mask = WD87480_HE.mask[::-1,::-1]

In [ ]:
WD85806_HE

In [ ]:
bg_img2 = WD85806_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img2[:, :, i][WD85806_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = WD85806_HE.show(RGB=True, mask_out=False)#, save_to="figures/show_WD87480_histology.png")

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/WD85806/WD85806_downsample8_7003_4_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/WD85806/WD85806_downsample8_7003_4_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
scalef_adata2 = a2.uns["spatial"]["7003_AS_4"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef2 = affine2[0,0]

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * (scalef_adata1*1.9)#(scalef_adata1/scalef1)

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 500
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 800

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 200
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 1000

In [ ]:
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 200
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 400

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 100
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 200

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img2,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD85806_HE.img.shape[1],0,WD85806_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD85806 Combined

In [ ]:
# flip coords in second sample
a2.obsm["spatial_ws"][:,0] = WD85806_HE.img.shape[1] - a2.obsm["spatial_ws"][:,0]
a2.obsm["spatial_ws"][:,1] = WD85806_HE.img.shape[0] - a2.obsm["spatial_ws"][:,1]

In [ ]:
a_ws = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("PAT73458_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a_ws = sc.read("PAT73458_combined_ws.h5ad")

In [ ]:
del a_ws.obs

In [ ]:
a1 = sc.read("7003_3_WD85806_master.h5ad")
a2 = sc.read("7003_4_WD85806_master.h5ad")

In [ ]:
a = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["cnv_leiden"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD85806_HE.img.shape[1],0,WD85806_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT73458_cnv_leiden.png"
)

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=signatures,
    ncols=5,
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD85806_HE.img.shape[1],0,WD85806_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT73458_signatures.png"
)

In [ ]:
et = ['A1_Innate', 'A2_Cytotoxic', 'A3_Endocrine', 'A4_MUC', 'A5_ABS', 'A6_SM', 'A7_Adaptive', 'A8_Stem']

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=et,
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD85806_HE.img.shape[1],0,WD85806_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT73458_archetypes.png",
    ncols=4,
)

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["CytoTRACE","cnv_score","cnv_leiden"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD85806_HE.img.shape[1],0,WD85806_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT73458_cnv_progression.png",
)

---
# WD87478 Sample 7003-AS-1

In [ ]:
a1 = sc.read("7003_1_WD87478_master.h5ad"); a1

In [ ]:
WD87478 = img.from_npz(file="../data/MxIF/WD87478/WD87478_downsample8.npz"); WD87478

In [ ]:
img1 = io.imread("../data/MxIF/WD87478/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
img1.shape

In [ ]:
WD87478_HE = img(img1, mask=WD87478.mask)
WD87478_HE.scale()
WD87478_HE.img = WD87478_HE.img[::-1,::-1,:]
WD87478_HE.mask = WD87478_HE.mask[::-1,::-1]

In [ ]:
WD87478_HE

In [ ]:
bg_img = WD87478_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD87478_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = WD87478_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT74143_histology.png")

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/WD87478/WD87478_downsample8_7003_1_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/WD87478/WD87478_downsample8_7003_1_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
scalef_adata1 = a1.uns["spatial"]["7003_AS_1"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef_adata1

In [ ]:
scalef1 = affine1[0,0]

In [ ]:
scalef1

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * (scalef_adata1*2.2)#(scalef_adata1/scalef1)

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 600
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 4350

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 100
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 50

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 200
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 50

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 100
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 50

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 50
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 50

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="pathology_annotation",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87478_HE.img.shape[1],0,WD87478_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD87478 Sample 7003-AS-2

In [ ]:
a2 = sc.read("7003_2_WD87478_master.h5ad"); a2

In [ ]:
img1 = io.imread("../data/MxIF/WD87478/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
WD87478_HE = img(img1, mask=WD87478.mask)
WD87478_HE.scale()
WD87478_HE.img = WD87478_HE.img[::-1,::-1,:]
WD87478_HE.mask = WD87478_HE.mask[::-1,::-1]

In [ ]:
WD87478_HE

In [ ]:
bg_img2 = WD87478_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img2[:, :, i][WD87478_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/WD87478/WD87478_downsample8_7003_2_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/WD87478/WD87478_downsample8_7003_2_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
scalef_adata2 = a2.uns["spatial"]["7003_AS_2"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef2 = affine2[0,0]

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * (scalef_adata1*2.2)#(scalef_adata1/scalef1)

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 1000
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 800

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 600
#a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 200

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img2,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87478_HE.img.shape[1],0,WD87478_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD87478 Combined

In [ ]:
a_ws = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("PAT74143_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a_ws = sc.read("PAT74143_combined_ws.h5ad")

In [ ]:
del a_ws.obs

In [ ]:
a1 = sc.read("7003_1_WD87478_master.h5ad")
a2 = sc.read("7003_2_WD87478_master.h5ad")

In [ ]:
a = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["cnv_leiden"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87478_HE.img.shape[1],0,WD87478_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT74143_cnv_leiden.png"
)

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["CytoTRACE","cnv_score","cnv_leiden"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87478_HE.img.shape[1],0,WD87478_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT74143_cnv_progression.png",
)

In [ ]:
et = ['A1_Innate', 'A2_Cytotoxic', 'A3_Endocrine', 'A4_MUC', 'A5_ABS', 'A6_SM', 'A7_Adaptive', 'A8_Stem']

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=et,
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87478_HE.img.shape[1],0,WD87478_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT74143_archetypes.png",
    ncols=4,
)

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=signatures,
    ncols=5,
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD87478_HE.img.shape[1],0,WD87478_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT74143_signatures.png"
)

---
# WD84602 Sample 8270-AS-6

In [ ]:
a1 = sc.read("../data/ST/8270_6_WD84602_master.h5ad"); a1

In [ ]:
WD84602 = img.from_npz(file="../data/MxIF/WD84602/WD84602_downsample8.npz"); WD84602

In [ ]:
img1 = io.imread("../data/MxIF/WD84602/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
WD84602_HE = img(img1, mask=WD84602.mask)
WD84602_HE.scale()
#WD84602_HE.img = WD84602_HE.img[::-1,::-1,:]
#WD84602_HE.mask = WD84602_HE.mask[::-1,::-1]

In [ ]:
WD84602_HE

In [ ]:
bg_img = WD84602_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD84602_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = WD84602_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT30884_histology.png")

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/WD84602/WD84602_downsample8_8270_6_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/WD84602/WD84602_downsample8_8270_6_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
shape1 * affine1[0,0]

In [ ]:
scalef_adata1 = a1.uns["spatial"]["8270_AS_6"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef1 = affine1[0,0]

In [ ]:
scalef_adata1

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * 0.8

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 200
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 900

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 2500
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 300

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 1000
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 200

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 400
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 200

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 100
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 300

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="CNV Score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD84602_HE.img.shape[1],0,WD84602_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD84602 Sample 8270-AS-7

In [ ]:
a2 = sc.read("../data/ST/8270_7_WD84602_master.h5ad"); a2

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/WD84602/WD84602_downsample8_8270_7_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/WD84602/WD84602_downsample8_8270_7_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * 0.8

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 2500
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 500

In [ ]:
#a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 2500
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 1500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 1500
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 500
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 200
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 100
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2500

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD84602_HE.img.shape[1],0,WD84602_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD84602 Combined

In [ ]:
a_ws = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("../data/ST/PAT30884_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a_ws = sc.read("../data/ST/PAT30884_combined_ws.h5ad")

In [ ]:
del a_ws.obs

In [ ]:
a1 = sc.read("../data/ST/8270_6_WD84602_master.h5ad")
a2 = sc.read("../data/ST/8270_7_WD84602_master.h5ad")

In [ ]:
a = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["CNV Clone"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD84602_HE.img.shape[1],0,WD84602_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT30884_cnv_leiden.png"
)

---
# WD84216 Sample 8270-AS-11

In [ ]:
a1 = sc.read("../data/ST/8270_11_WD84216_master.h5ad"); a1

In [ ]:
WD84216 = img.from_npz(file="../data/MxIF/WD84216/WD84216_downsample8.npz"); WD84216

In [ ]:
img1 = io.imread("../data/MxIF/WD84216/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
WD84216_HE = img(img1, mask=WD84216.mask)
WD84216_HE.scale()
WD84216_HE.img = WD84216_HE.img[::-1,::-1,:]
WD84216_HE.mask = WD84216_HE.mask[::-1,::-1]

In [ ]:
WD84216_HE

In [ ]:
bg_img = WD84216_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD84216_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = WD84216_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT54273_histology.png")

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/WD84216/WD84216_downsample8_8270_11_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/WD84216/WD84216_downsample8_8270_11_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * 0.8

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 2000
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 500

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 3000
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 300

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 1500
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] - 500

In [ ]:
#a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 400
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] - 200

In [ ]:
#a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 400
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] - 150

In [ ]:
#a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 400
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] - 100

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="CNV Score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD84216_HE.img.shape[1],0,WD84216_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD84216 Sample 8270-AS-12

In [ ]:
a2 = sc.read("../data/ST/8270_12_WD84216_master.h5ad"); a2

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/WD84216/WD84216_downsample8_8270_12_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/WD84216/WD84216_downsample8_8270_12_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * 0.8

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 500
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 2000

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 500
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 2000

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 500
#a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 2000

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 300
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 100

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 100
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 50

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD84216_HE.img.shape[1],0,WD84216_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# WD84216 Combined

In [ ]:
a_ws = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("../data/ST/PAT54273_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a_ws = sc.read("../data/ST/PAT54273_combined_ws.h5ad")

In [ ]:
del a_ws.obs

In [ ]:
a1 = sc.read("../data/ST/8270_11_WD84216_master.h5ad")
a2 = sc.read("../data/ST/8270_12_WD84216_master.h5ad")

In [ ]:
a = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["CNV Clone"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD84216_HE.img.shape[1],0,WD84216_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_PAT54273_cnv_leiden.png"
)

---
# PAT33430 Block WD84221 Sample 7794-KL-3

In [ ]:
a = sc.read("../data/ST/7794_3_WD84221_master.h5ad"); a

In [ ]:
WD84221 = img.from_npz(file="../data/MxIF/WD84221/WD84221_downsample8.npz"); WD84221

In [ ]:
img1 = io.imread("../data/MxIF/WD84221/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
WD84221_HE = img(img1, mask=WD84221.mask)
WD84221_HE.scale()
#WD84221_HE.img = WD84221_HE.img[::-1,::-1,:]
#WD84221_HE.mask = WD84221_HE.mask[::-1,::-1]

In [ ]:
p = WD84221_HE.show(RGB=True)

In [ ]:
bg_img = WD84221_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD84221_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
affine4 = np.genfromtxt(
    "../data/MxIF/WD84221/WD84221_downsample8_7794_3_affine.csv",
    delimiter=",",
)
shape4 = np.genfromtxt(
    "../data/MxIF/WD84221/WD84221_downsample8_7794_3_shape.csv",
    delimiter=",",
)

In [ ]:
affine4

In [ ]:
shape4

In [ ]:
scalef_adata4 = a.uns["spatial"]["7794_AS_3"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef4 = affine4[0,0]

In [ ]:
scalef4

In [ ]:
scalef_adata4

In [ ]:
a.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a.obsm["spatial"]),
        np.zeros((a.obsm["spatial"].shape[0],1))
    ]),
    affine4.T
)[:,:2]

In [ ]:
a.obsm["spatial_ws"] = a.obsm["spatial_ws"] * 0.45

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 2000
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] - 500

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 1000
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 500

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] - 500
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 500

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] - 50
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 50

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] - 25
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 25

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] - 25
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 25

In [ ]:
#a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] - 25
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 50

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 200
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] - 100

In [ ]:
#a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] - 25
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] - 50

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 25
#a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] - 50

In [ ]:
sc.pl.spatial(
    a,
    basis="spatial_ws",
    color="CNV Score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD84221_HE.img.shape[1],0,WD84221_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

In [ ]:
p = WD84221_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT33430_histology.png")

In [ ]:
a.write("PAT33430_WD84221_combined_ws.h5ad", compression="gzip")

---
# PAT59460 Block WD84226 Sample 8270-AS-10

In [ ]:
a = sc.read("../data/ST/8270_10_WD84226_master.h5ad"); a

In [ ]:
WD84226 = img.from_npz(file="../data/MxIF/WD84226/WD84226_downsample8.npz"); WD84226

In [ ]:
img1 = io.imread("../data/MxIF/WD84226/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
WD84226_HE = img(img1, mask=WD84226.mask)
WD84226_HE.scale()
WD84226_HE.img = WD84226_HE.img[::-1,::-1,:]
WD84226_HE.mask = WD84226_HE.mask[::-1,::-1]

In [ ]:
p = WD84226_HE.show(RGB=True)

In [ ]:
bg_img = WD84226_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD84226_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
affine4 = np.genfromtxt(
    "../data/MxIF/WD84226/WD84226_downsample8_8270_10_affine.csv",
    delimiter=",",
)
shape4 = np.genfromtxt(
    "../data/MxIF/WD84226/WD84226_downsample8_8270_10_shape.csv",
    delimiter=",",
)

In [ ]:
affine4

In [ ]:
shape4

In [ ]:
scalef_adata4 = a.uns["spatial"]["8270_AS_10"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef4 = affine4[0,0]

In [ ]:
scalef4

In [ ]:
scalef_adata4

In [ ]:
a.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a.obsm["spatial"]),
        np.zeros((a.obsm["spatial"].shape[0],1))
    ]),
    affine4.T
)[:,:2]

In [ ]:
a.obsm["spatial_ws"] = a.obsm["spatial_ws"] * 0.85

In [ ]:
a.obsm["spatial_ws"]

In [ ]:
#a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 2000
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 2000

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] - 500
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 1000

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 200
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 1000

In [ ]:
#a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 200
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 1000

In [ ]:
#a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 200
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] - 200

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 300
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] - 100

In [ ]:
sc.pl.spatial(
    a,
    basis="spatial_ws",
    color="CNV Score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD84226_HE.img.shape[1],0,WD84226_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

In [ ]:
p = WD84226_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT59460_histology.png")

In [ ]:
a.write("PAT59460_WD84226_combined_ws.h5ad", compression="gzip")

---
# PAT59667 Block WD84596 Sample 8270-AS-8

In [ ]:
a = sc.read("../data/ST/8270_8_WD84596_master.h5ad"); a

In [ ]:
WD84596 = img.from_npz(file="../data/MxIF/WD84596/WD84596_downsample8.npz"); WD84596

In [ ]:
img1 = io.imread("../data/MxIF/WD84596/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
WD84596_HE = img(img1, mask=WD84596.mask)
WD84596_HE.scale()
WD84596_HE.img = WD84596_HE.img[::-1,::-1,:]
WD84596_HE.mask = WD84596_HE.mask[::-1,::-1]

In [ ]:
p = WD84596_HE.show(RGB=True)

In [ ]:
bg_img = WD84596_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD84596_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
affine4 = np.genfromtxt(
    "../data/MxIF/WD84596/WD84596_downsample8_8270_8_affine.csv",
    delimiter=",",
)
shape4 = np.genfromtxt(
    "../data/MxIF/WD84596/WD84596_downsample8_8270_8_shape.csv",
    delimiter=",",
)

In [ ]:
affine4

In [ ]:
shape4

In [ ]:
scalef_adata4 = a.uns["spatial"]["8270_AS_8"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef4 = affine4[0,0]

In [ ]:
scalef4

In [ ]:
scalef_adata4

In [ ]:
a.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a.obsm["spatial"]),
        np.zeros((a.obsm["spatial"].shape[0],1))
    ]),
    affine4.T
)[:,:2]

In [ ]:
a.obsm["spatial_ws"] = a.obsm["spatial_ws"] * 0.77

In [ ]:
a.obsm["spatial_ws"]

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 3000
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 4500

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 200
#a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 4500

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 100
#a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 4500

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] - 150
#a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 4500

In [ ]:
sc.pl.spatial(
    a,
    basis="spatial_ws",
    color="CNV Score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD84596_HE.img.shape[1],0,WD84596_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

In [ ]:
p = WD84596_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT59667_histology.png")

In [ ]:
a.write("PAT59667_WD84596_combined_ws.h5ad", compression="gzip")

---
# PAT59600 Block WD84594 Sample 8270-AS-9

In [ ]:
a = sc.read("../data/ST/8270_9_WD84594_master.h5ad"); a

In [ ]:
WD84594 = img.from_npz(file="../data/MxIF/WD84594/WD84594_downsample8.npz"); WD84594

In [ ]:
img1 = io.imread("../data/MxIF/WD84594/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
WD84594_HE = img(img1, mask=WD84594.mask)
WD84594_HE.scale()
#WD84594_HE.img = WD84594_HE.img[::-1,::-1,:]
#WD84594_HE.mask = WD84594_HE.mask[::-1,::-1]

In [ ]:
p = WD84594_HE.show(RGB=True)

In [ ]:
bg_img = WD84594_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][WD84594_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
affine4 = np.genfromtxt(
    "../data/MxIF/WD84594/WD84594_downsample8_8270_9_affine.csv",
    delimiter=",",
)
shape4 = np.genfromtxt(
    "../data/MxIF/WD84594/WD84594_downsample8_8270_9_shape.csv",
    delimiter=",",
)

In [ ]:
affine4

In [ ]:
shape4

In [ ]:
scalef_adata4 = a.uns["spatial"]["8270_AS_9"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef4 = affine4[0,0]

In [ ]:
scalef4

In [ ]:
scalef_adata4

In [ ]:
a.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a.obsm["spatial"]),
        np.zeros((a.obsm["spatial"].shape[0],1))
    ]),
    affine4.T
)[:,:2]

In [ ]:
a.obsm["spatial_ws"] = a.obsm["spatial_ws"] * 0.8

In [ ]:
a.obsm["spatial_ws"]

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 2000
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] - 1000

In [ ]:
a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 500
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] - 500

In [ ]:
#a.obsm["spatial_ws"][:,0] = a.obsm["spatial_ws"][:,0] + 500
a.obsm["spatial_ws"][:,1] = a.obsm["spatial_ws"][:,1] + 1000

In [ ]:
sc.pl.spatial(
    a,
    basis="spatial_ws",
    color="CNV Score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,WD84594_HE.img.shape[1],0,WD84594_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

In [ ]:
p = WD84594_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_PAT59600_histology.png")

In [ ]:
a.write("PAT59600_WD84594_combined_ws.h5ad", compression="gzip")

---
# SG00001 Sample 8899-AS-1

In [ ]:
a1 = sc.read("../data/ST/8899_1_SG00001_master.h5ad"); a1

In [ ]:
SG00001 = img.from_npz(file="../data/MxIF/SG00001/SG00001_downsample8.npz"); SG00001

In [ ]:
img1 = io.imread("../data/MxIF/SG00001/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
SG00001_HE = img(img1, mask=SG00001.mask)
SG00001_HE.scale()
#WD84602_HE.img = WD84602_HE.img[::-1,::-1,:]
#WD84602_HE.mask = WD84602_HE.mask[::-1,::-1]

In [ ]:
SG00001

In [ ]:
bg_img = SG00001_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][SG00001_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = SG00001_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_SG00001_histology.png")

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/SG00001/SG00001_downsample8_8899_1_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/SG00001/SG00001_downsample8_8899_2_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
shape1 * affine1[0,0]

In [ ]:
scalef_adata1 = a1.uns["spatial"]["8899_AS_1"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef1 = affine1[0,0]

In [ ]:
scalef_adata1

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * 0.8

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 200
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 3500

In [ ]:
#a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 200
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 5500

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 200
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 5500

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 500
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 5500

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="CNV Score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00001_HE.img.shape[1],0,SG00001_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# SG00001 Sample 8899-AS-2

In [ ]:
a2 = sc.read("../data/ST/8899_2_SG00001_master.h5ad"); a2

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/SG00001/SG00001_downsample8_8899_2_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/SG00001/SG00001_downsample8_8899_2_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * 0.8

In [ ]:
#a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 2000
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2000

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 2500
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 8000

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 800
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 800

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 500
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 200
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 100
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 100

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 100
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2500

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00001_HE.img.shape[1],0,SG00001_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# SG00001 Combined

In [ ]:
a_ws = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("../data/ST/SG00001_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a_ws = sc.read("../data/ST/SG00001_combined_ws.h5ad")

In [ ]:
del a_ws.obs

In [ ]:
a1 = sc.read("../data/ST/8899_1_SG00001_master.h5ad")
a2 = sc.read("../data/ST/8899_2_SG00001_master.h5ad")

In [ ]:
a = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["CNV Clone"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00001_HE.img.shape[1],0,SG00001_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_SG00001_cnv_leiden.png"
)

---
# SG00002 Sample 8899-AS-3

In [ ]:
a1 = sc.read("../data/ST/8899_3_SG00002_master.h5ad"); a1

In [ ]:
SG00002 = img.from_npz(file="../data/MxIF/SG00002/SG00002_downsample8.npz"); SG00002

In [ ]:
img1 = io.imread("../data/MxIF/SG00002/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
SG00002_HE = img(img1, mask=SG00002.mask)
SG00002_HE.scale()
#WD84602_HE.img = WD84602_HE.img[::-1,::-1,:]
#WD84602_HE.mask = WD84602_HE.mask[::-1,::-1]

In [ ]:
SG00002

In [ ]:
bg_img = SG00002_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][SG00002_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = SG00002_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_SG00002_histology.png")

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/SG00002/SG00002_downsample8_8899_3_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/SG00002/SG00002_downsample8_8899_3_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
shape1 * affine1[0,0]

In [ ]:
scalef_adata1 = a1.uns["spatial"]["8899_AS_3"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef1 = affine1[0,0]

In [ ]:
scalef_adata1

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * 0.8

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 2500
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 300

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 4000
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 4000

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 300
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] - 200

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 100
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 300

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="CNV Score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00002_HE.img.shape[1],0,SG00002_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# SG00002 Sample 8899-AS-4

In [ ]:
a2 = sc.read("../data/ST/8899_4_SG00002_master.h5ad"); a2

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/SG00002/SG00002_downsample8_8899_4_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/SG00002/SG00002_downsample8_8899_4_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * 0.8

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2000
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 3000

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 8000
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 500

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 500
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 600

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 100
#a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 600

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 100
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 50

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 100
#a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 600

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00002_HE.img.shape[1],0,SG00002_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# SG00002 Combined

In [ ]:
a_ws = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("../data/ST/SG00002_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a_ws = sc.read("../data/ST/SG00002_combined_ws.h5ad")

In [ ]:
del a_ws.obs

In [ ]:
a1 = sc.read("../data/ST/8899_3_SG00002_master.h5ad")
a2 = sc.read("../data/ST/8899_4_SG00002_master.h5ad")

In [ ]:
a = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["CNV Clone"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00002_HE.img.shape[1],0,SG00002_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_SG00002_cnv_leiden.png"
)

---
# SG00003 Sample 8899-AS-5

In [ ]:
a1 = sc.read("../data/ST/8899_5_SG00003_master.h5ad"); a1

In [ ]:
SG00003 = img.from_npz(file="../data/MxIF/SG00003/SG00003_downsample8.npz"); SG00003

In [ ]:
img1 = io.imread("../data/MxIF/SG00003/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
SG00003_HE = img(img1, mask=SG00003.mask)
SG00003_HE.scale()
#WD84602_HE.img = WD84602_HE.img[::-1,::-1,:]
#WD84602_HE.mask = WD84602_HE.mask[::-1,::-1]

In [ ]:
SG00003

In [ ]:
bg_img = SG00003_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][SG00003_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = SG00003_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_SG00003_histology.png")

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/SG00003/SG00003_downsample8_8899_5_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/SG00003/SG00003_downsample8_8899_5_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
shape1 * affine1[0,0]

In [ ]:
scalef_adata1 = a1.uns["spatial"]["8899_AS_5"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef1 = affine1[0,0]

In [ ]:
scalef_adata1

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * 0.8

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 900
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 500

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 2000
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 2000

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 600
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 800

In [ ]:
#a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 400
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 300

In [ ]:
#a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 100
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 80

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="CNV Score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00003_HE.img.shape[1],0,SG00003_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# SG00003 Sample 8899-AS-6

In [ ]:
a2 = sc.read("../data/ST/8899_6_SG00003_master.h5ad"); a2

In [ ]:
SG00003_HE = img(img1, mask=SG00003.mask)
SG00003_HE.scale()
SG00003_HE.img = SG00003_HE.img[::-1,::-1,:]
SG00003_HE.mask = SG00003_HE.mask[::-1,::-1]

In [ ]:
bg_img = SG00003_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][SG00003_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/SG00003/SG00003_downsample8_8899_6_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/SG00003/SG00003_downsample8_8899_6_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * 0.8

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 500
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2000

In [ ]:
#a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 2500
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 1500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 200
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 500

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 50
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 50

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 4000
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 5000

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 3000
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 5000

In [ ]:
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 100
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2500

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00003_HE.img.shape[1],0,SG00003_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# SG00003 Combined

In [ ]:
# flip coords in second sample
a2.obsm["spatial_ws"][:,0] = SG00003_HE.img.shape[1] - a2.obsm["spatial_ws"][:,0]
a2.obsm["spatial_ws"][:,1] = SG00003_HE.img.shape[0] - a2.obsm["spatial_ws"][:,1]

In [ ]:
a_ws = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("../data/ST/SG00003_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a_ws = sc.read("../data/ST/SG00003_combined_ws.h5ad")

In [ ]:
del a_ws.obs

In [ ]:
a1 = sc.read("../data/ST/8899_5_SG00003_master.h5ad")
a2 = sc.read("../data/ST/8899_6_SG00003_master.h5ad")

In [ ]:
a = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
SG00003_HE = img(img1, mask=SG00003.mask)
SG00003_HE.scale()

In [ ]:
bg_img = SG00003_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][SG00003_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["CNV Clone"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00003_HE.img.shape[1],0,SG00003_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_SG00003_cnv_leiden.png"
)

---
# SG00004 Sample 8899-AS-7

In [ ]:
a1 = sc.read("../data/ST/8899_7_SG00004_master.h5ad"); a1

In [ ]:
SG00004 = img.from_npz(file="../data/MxIF/SG00004/SG00004_downsample8.npz"); SG00004

In [ ]:
img1 = io.imread("../data/MxIF/SG00004/VirtualStains/S002_VHE_stitched_downsample8.tif")

In [ ]:
SG00004_HE = img(img1, mask=SG00004.mask)
SG00004_HE.scale()
#WD84602_HE.img = WD84602_HE.img[::-1,::-1,:]
#WD84602_HE.mask = WD84602_HE.mask[::-1,::-1]

In [ ]:
SG00004

In [ ]:
bg_img = SG00004_HE.img
for i in [0, 1, 2]:  # for 3-channel image
    bg_img[:, :, i][SG00004_HE.mask == 0] = 1.0  # area outside mask 1.0

In [ ]:
p = SG00004_HE.show(RGB=True, mask_out=False, save_to="plots_patient/wholeslide/show_SG00004_histology.png")

In [ ]:
affine1 = np.genfromtxt(
    "../data/MxIF/SG00004/SG00004_downsample8_8899_7_affine.csv",
    delimiter=",",
)
shape1 = np.genfromtxt(
    "../data/MxIF/SG00004/SG00004_downsample8_8899_7_shape.csv",
    delimiter=",",
)

In [ ]:
affine1

In [ ]:
shape1

In [ ]:
shape1 * affine1[0,0]

In [ ]:
scalef_adata1 = a1.uns["spatial"]["8899_AS_7"]["scalefactors"]["tissue_hires_scalef"]

In [ ]:
scalef1 = affine1[0,0]

In [ ]:
scalef_adata1

In [ ]:
a1.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a1.obsm["spatial"]),
        np.zeros((a1.obsm["spatial"].shape[0],1))
    ]),
    affine1.T
)[:,:2]

In [ ]:
a1.obsm["spatial_ws"] = a1.obsm["spatial_ws"] * 0.9

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 2000
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 900

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 1000
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 2000

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 1000
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 1000

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] + 700
a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 700

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 400
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 700

In [ ]:
a1.obsm["spatial_ws"][:,0] = a1.obsm["spatial_ws"][:,0] - 100
#a1.obsm["spatial_ws"][:,1] = a1.obsm["spatial_ws"][:,1] + 700

In [ ]:
a1.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a1,
    basis="spatial_ws",
    color="CNV Score",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00004_HE.img.shape[1],0,SG00004_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# SG00004 Sample 8899-AS-8

In [ ]:
a2 = sc.read("../data/ST/8899_8_SG00004_master.h5ad"); a2

In [ ]:
affine2 = np.genfromtxt(
    "../data/MxIF/SG00004/SG00004_downsample8_8899_8_affine.csv",
    delimiter=",",
)
shape2 = np.genfromtxt(
    "../data/MxIF/SG00004/SG00004_downsample8_8899_8_shape.csv",
    delimiter=",",
)

In [ ]:
affine2

In [ ]:
shape2

In [ ]:
a2.obsm["spatial_ws"] = np.dot(
    np.column_stack([
        (a2.obsm["spatial"]),
        np.zeros((a2.obsm["spatial"].shape[0],1))
    ]),
    affine2.T
)[:,:2]

In [ ]:
a2.obsm["spatial_ws"] = a2.obsm["spatial_ws"] * 0.8

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 2500
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 500

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 7000
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 2000

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] + 700
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 1500

In [ ]:
a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 300
#a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] + 1500

In [ ]:
#a2.obsm["spatial_ws"][:,0] = a2.obsm["spatial_ws"][:,0] - 300
a2.obsm["spatial_ws"][:,1] = a2.obsm["spatial_ws"][:,1] - 150

In [ ]:
a2.obsm["spatial_ws"]

In [ ]:
sc.pl.spatial(
    a2,
    basis="spatial_ws",
    color="CytoTRACE",
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00004_HE.img.shape[1],0,SG00004_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
)

---
# SG00004 Combined

In [ ]:
a_ws = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws

In [ ]:
a_ws.write("../data/ST/SG00004_combined_ws.h5ad", compression="gzip")

---
### Replace old metadata with new

In [ ]:
a_ws = sc.read("../data/ST/SG00004_combined_ws.h5ad")

In [ ]:
del a_ws.obs

In [ ]:
a1 = sc.read("../data/ST/8899_7_SG00004_master.h5ad")
a2 = sc.read("../data/ST/8899_8_SG00004_master.h5ad")

In [ ]:
a = a1.concatenate([a2], fill_value=0)

In [ ]:
a_ws.obs = a_ws.obs.merge(a.obs, left_index=True, right_index=True)

---

In [ ]:
sc.pl.spatial(
    a_ws,
    basis="spatial_ws",
    color=["CNV Clone"],
    img=bg_img,
    spot_size=50,
    scale_factor=1,
    crop_coord=[0,SG00004_HE.img.shape[1],0,SG00004_HE.img.shape[0]],
    frameon=False,
    cmap="viridis",
    save="_SG00004_cnv_leiden.png"
)